# Chapter 02-03: Subclassing API

## 학습 목표
- `tf.keras.Model`을 상속하여 완전 커스텀 모델을 만든다
- `tf.keras.layers.Layer`를 상속하여 커스텀 레이어를 만든다
- `__init__`, `build`, `call` 메서드의 역할을 이해한다

## 목차
1. Subclassing API 개요
2. tf.keras.Model 상속 예시
3. 커스텀 레이어 만들기
4. add_weight로 가중치 직접 정의
5. Subclassing으로 MNIST 분류기 완성
6. 3가지 API 비교

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
print("TensorFlow 버전:", tf.__version__)

## Subclassing API

`tf.keras.Model`을 상속하여 완전히 커스텀 모델을 만든다.

**사용 시점**: 동적 구조, 연구용 특수 레이어, 조건부 연산이 필요할 때

**핵심 메서드**:
- `__init__`: 레이어 정의
- `call(inputs, training=False)`: 순전파 정의

## 2. tf.keras.Model 상속 예시

`__init__`에서 레이어를 **인스턴스 속성**으로 선언하고,
`call`에서 데이터 흐름을 직접 작성한다.

In [ ]:
class MNISTClassifier(tf.keras.Model):
    """MNIST 분류를 위한 Subclassing 모델"""

    def __init__(self, num_classes=10):
        super().__init__()  # 부모 클래스 초기화 (필수)

        # 레이어를 인스턴스 속성으로 정의
        self.flatten = tf.keras.layers.Flatten()              # 2D → 1D
        self.dense1  = tf.keras.layers.Dense(128, activation='relu')  # 은닉층 1
        self.dropout = tf.keras.layers.Dropout(0.2)          # 과적합 방지
        self.dense2  = tf.keras.layers.Dense(num_classes, activation='softmax')  # 출력층

    def call(self, inputs, training=False):
        """순전파 정의: training 인수로 Dropout 동작을 제어"""
        x = self.flatten(inputs)               # 이미지 펼치기
        x = self.dense1(x)                     # 은닉층 변환
        x = self.dropout(x, training=training) # 훈련 시에만 드롭아웃 적용
        return self.dense2(x)                  # 확률 분포 출력


# 모델 인스턴스 생성
subclass_model = MNISTClassifier(num_classes=10)

# 더미 데이터로 빌드 (summary를 보려면 입력 shape를 알아야 함)
dummy_input = tf.zeros((1, 28, 28))  # 배치 1개짜리 더미 이미지
_ = subclass_model(dummy_input)       # 한 번 호출하여 가중치 초기화

subclass_model.summary()
print(f"\n파라미터 수: {subclass_model.count_params():,}")

## 3. 커스텀 레이어 만들기

`tf.keras.layers.Layer`를 상속하면 재사용 가능한 커스텀 레이어를 만들 수 있다.

- `build(input_shape)`: 입력 shape가 확정된 시점에 **가중치 생성** (지연 초기화)
- `call(inputs)`: 레이어 연산 정의

In [ ]:
class ResidualBlock(tf.keras.layers.Layer):
    """커스텀 잔차 블록 레이어
    
    h = ReLU(Dense(Dense(x)) + x)
    """

    def __init__(self, units, **kwargs):
        super().__init__(**kwargs)  # name 등 공통 인수 전달
        self.units = units

        # 블록 내부 레이어 정의
        self.dense1 = tf.keras.layers.Dense(units, activation='relu')
        self.dense2 = tf.keras.layers.Dense(units, activation=None)  # Add 전에는 활성화 없음
        self.relu   = tf.keras.layers.Activation('relu')

    def build(self, input_shape):
        """입력 shape 확정 후 가중치 초기화 (자동 호출)"""
        # 내부 레이어들은 첫 call 때 자동으로 build됨
        super().build(input_shape)  # built = True 플래그 설정

    def call(self, inputs):
        """잔차 연결: h = ReLU(F(x) + x)"""
        h = self.dense1(inputs)           # 첫 번째 변환
        h = self.dense2(h)                # 두 번째 변환 (활성화 없음)
        return self.relu(h + inputs)      # 스킵 연결 후 활성화

    def get_config(self):
        """직렬화를 위한 설정 반환 (모델 저장 시 필요)"""
        config = super().get_config()
        config.update({'units': self.units})
        return config


# 커스텀 레이어 테스트
x_test = tf.random.normal((4, 64))   # 배치 4, 차원 64
res_block = ResidualBlock(units=64, name='test_res_block')
y_test = res_block(x_test)
print("입력 shape:", x_test.shape)
print("출력 shape:", y_test.shape)   # 잔차 연결이므로 동일해야 함
print("ResidualBlock 파라미터 수:", res_block.count_params())

## 4. add_weight로 가중치 직접 정의

`add_weight`를 사용하면 Dense 레이어 없이도 원하는 가중치를 직접 생성할 수 있다.
이는 완전히 새로운 연산(ex. 어텐션, 커스텀 정규화)을 구현할 때 유용하다.

In [ ]:
class CustomDenseLayer(tf.keras.layers.Layer):
    """add_weight로 Dense 레이어를 직접 구현한 예시"""

    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        # 활성화 함수를 문자열 또는 callable로 처리
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        """입력 shape를 알아야 W의 크기를 결정할 수 있으므로 build에서 생성"""
        n_in = int(input_shape[-1])  # 마지막 차원 = 입력 특징 수

        # 가중치 행렬 W: shape (n_in, n_out)
        self.W = self.add_weight(
            name='kernel',
            shape=(n_in, self.units),
            initializer='glorot_uniform',   # Xavier 초기화
            trainable=True                  # 역전파로 업데이트됨
        )

        # 편향 벡터 b: shape (n_out,)
        self.b = self.add_weight(
            name='bias',
            shape=(self.units,),
            initializer='zeros',            # 편향은 0으로 초기화
            trainable=True
        )

        super().build(input_shape)  # built = True

    def call(self, inputs):
        """y = activation(x @ W + b)"""
        z = tf.matmul(inputs, self.W) + self.b  # 선형 변환
        return self.activation(z)               # 활성화 함수 적용

    def get_config(self):
        config = super().get_config()
        config.update({
            'units': self.units,
            'activation': tf.keras.activations.serialize(self.activation)
        })
        return config


# 커스텀 Dense 레이어 테스트 및 내장 Dense와 비교
custom_dense = CustomDenseLayer(units=32, activation='relu')
builtin_dense = tf.keras.layers.Dense(units=32, activation='relu')

x_sample = tf.random.normal((8, 64))  # 배치 8, 입력 64차원

out_custom  = custom_dense(x_sample)
out_builtin = builtin_dense(x_sample)

print("커스텀 Dense 출력 shape:", out_custom.shape)
print("내장 Dense 출력 shape:",   out_builtin.shape)
print("커스텀 Dense 파라미터:",   custom_dense.count_params())
print("내장 Dense 파라미터:",     builtin_dense.count_params())

## 5. Subclassing으로 MNIST 분류기 완성

In [ ]:
class DeepMNISTClassifier(tf.keras.Model):
    """커스텀 잔차 블록을 사용한 MNIST 분류기"""

    def __init__(self, num_classes=10):
        super().__init__()
        self.flatten    = tf.keras.layers.Flatten()
        self.projection = tf.keras.layers.Dense(64, activation='relu')  # 차원 투영
        self.res_block1 = ResidualBlock(units=64, name='res_1')          # 잔차 블록 1
        self.res_block2 = ResidualBlock(units=64, name='res_2')          # 잔차 블록 2
        self.dropout    = tf.keras.layers.Dropout(0.3)
        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs, training=False):
        x = self.flatten(inputs)
        x = self.projection(x)                      # 64차원으로 투영
        x = self.res_block1(x)                      # 첫 번째 잔차 블록
        x = self.res_block2(x)                      # 두 번째 잔차 블록
        x = self.dropout(x, training=training)      # 훈련 시에만 드롭아웃
        return self.classifier(x)


# 모델 생성 및 컴파일
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # 정규화

deep_model = DeepMNISTClassifier(num_classes=10)
deep_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 더미 입력으로 빌드 후 summary 출력
_ = deep_model(tf.zeros((1, 28, 28)))
deep_model.summary()

# 학습
history = deep_model.fit(
    x_train, y_train,
    epochs=5,
    batch_size=128,
    validation_split=0.1,
    verbose=1
)

# 테스트 평가
test_loss, test_acc = deep_model.evaluate(x_test, y_test, verbose=0)
print(f"\n테스트 정확도: {test_acc:.4f}")

## 6. 3가지 API 비교

| 항목 | Sequential | Functional | Subclassing |
|------|-----------|------------|-------------|
| 유연성 | 낮음 | 중간 | 높음 |
| 모델 시각화 | 가능 | 가능 | 제한적 |
| 동적 구조 | 불가 | 불가 | 가능 |
| 디버깅 | 쉬움 | 쉬움 | 중간 |
| 재사용성 | 낮음 | 중간 | 높음 |
| 권장 상황 | 튜토리얼, 프로토타입 | 복잡한 구조 | 연구, 특수 레이어 |

**다음**: 04_layers_and_activations.ipynb — 다양한 레이어와 활성화 함수